# News Article Summarizer

### Workflow for Building a News Articles Summarizer with Bulleted Lists
In this project, we are going to summarize a news article using Few Shot Prompting techniques and Output Parsing techniques. 

![Alt text](image.png)

Load the article from the "newspaper" library

In [4]:
# Import necessary libraries
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()

try:
  response = session.get(article_url, headers=headers, timeout=10)
  
  if response.status_code == 200:
      article = Article(article_url)
      article.download()
      article.parse()
      
      print(f"Title: {article.title}")
      print(f"Text: {article.text}")
  else:
      print(f"Failed to fetch article at {article_url}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_url}: {e}")



Title: Meta claims its new AI supercomputer will set records
Text: Ryan is a senior editor at TechForge Media with over a decade of experience covering the latest technology and interviewing leading industry figures. He can often be sighted at tech conferences with a strong coffee in one hand and a laptop in the other. If it's geeky, he’s probably into it. Find him on Twitter (@Gadget_Ry) or Mastodon (@gadgetry@techhub.social)

Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and the aim is for it to be capable of training models with trillions of parameters.

“We hope RSC will help us build entire

### Option 1: Generating examples in the prompt

Using a prompt with examples to generate the summary

In [6]:
from langchain.schema import (
    HumanMessage
)


# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """
As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:

Example 1:
Original Article: 'The Effects of Climate Change
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather conditions.

Example 2:
Original Article: 'The Evolution of Artificial Intelligence
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

==================
Title: {article_title}

{article_text}
==================

Please provide a summarized version of the article in a bulleted list format.
"""

# Format the Prompt
prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]


# Generate model and summary 


from langchain.chat_models import ChatOpenAI

# load the model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

# generate summary
summary = chat(messages)
print(summary.content)

- Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC).
- The RSC is expected to be the world's fastest supercomputer once complete.
- It will be capable of training models with trillions of parameters.
- Meta aims to use the RSC to build new AI systems for real-time voice translations and applications in the metaverse.
- The RSC is estimated to be 20x faster than Meta's current clusters and 9x faster at running the NVIDIA Collective Communication Library (NCCL).
- It will also be 3x faster at training large-scale natural language processing (NLP) workflows.
- Meta designed the RSC with security and privacy controls to use real-world examples from its production systems.
- This will allow Meta to advance research for tasks such as identifying harmful content on its platforms.
- The RSC is expected to significantly improve training times for models with tens of billions of parameters.


### Option 2: Using FewShotPromptTemplate

Uing the FewShotPomptTemplate class to construct the prompt with examples

In [ ]:
from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

# we get the article data from the scraping part
article_title = article.title
article_text = article.text

llm = OpenAI(model_name="text-davinci-003", temperature=0)

examples=[
        {
            "title": "The Effects of Climate Change",
            "summary": 
                """* Climate change is causing a rise in global temperatures.
                * This leads to melting ice caps and rising sea levels.
                * Resulting in more frequent and severe weather conditions."""

        },
        {
            "title": "The Evolution of Artificial Intelligence",
            "summary": 
                """* Artificial Intelligence (AI) has developed significantly over the past decade.
                * AI is now used in multiple fields such as healthcare, finance, and transportation.
                * The future of AI is promising but requires careful regulation."""

        }
    ]

example_template = """
User: {title}
AI: {summary}
"""

example_prompt = PromptTemplate(
    input_variables=["title", "summary"],
    template=example_template
)

prefix = """As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:
"""

suffix = """
User: {title}
Article :{article}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["title","article"],
    example_separator="\n\n"
)

# Create the LLMChain for the few_shot_prompt_template
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# Run the LLMChain with input_data
input_data = {"title": article_title, "article":article_text}
response = chain.run(input_data)


print(response)

/Users/maitemarzoratti/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain/llms/openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/maitemarzoratti/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain/llms/openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


* Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC).
* The RSC is set to be the world's fastest supercomputer once complete.
* It will be capable of training models with trillions of parameters.
* Meta hopes that the RSC will enable the development of new AI systems for real-time voice translations and collaboration in the metaverse.
* The RSC is expected to be 20x faster than Meta's current clusters and 9x faster at running the NVIDIA Collective Communication Library (NCCL).
* It will also be 3x faster at training large-scale natural language processing (NLP) workflows.
* The RSC allows Meta to use real-world examples from its production systems for research and training.
* Meta believes that the RSC is the first infrastructure to tackle performance, reliability, security, and privacy at such a scale.


### Option 3: Parsing the output

Using an output parser to format the output into the desired format.

In [7]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator
from pydantic import BaseModel, Field
from typing import List


# create output parser class
class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")

    # validating whether the generated summary has at least three lines
    @validator('summary', allow_reuse=True)
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines

# set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

In [8]:
from langchain.prompts import PromptTemplate


# create prompt template
# notice that we are specifying the "partial_variables" parameter
template = """
You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Format the prompt using the article title and text obtained from scraping
formatted_prompt = prompt.format_prompt(article_title=article_title, article_text=article_text)

In [9]:
from langchain.llms import OpenAI

# instantiate model class with temperature 0 to be deterministic and favor the most likely outcome over randomness/creativity
model = OpenAI(model_name="text-davinci-003", temperature=0.0)

# Use the model to generate a summary
output = model(formatted_prompt.to_string())

# Parse the output into the Pydantic model
parsed_output = parser.parse(output)
print(parsed_output)

title='Meta claims its new AI supercomputer will set records' summary=['Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that it claims will be the world’s fastest.', 'RSC is set to be fully built in mid-2022 and is expected to be capable of training models with trillions of parameters.', 'Meta expects RSC will be 20x faster than its current V100-based clusters and is estimated to be 9x faster at running the NVIDIA Collective Communication Library (NCCL) and 3x faster at training large-scale NLP workflows.', 'RSC was designed with security and privacy controls in mind to allow Meta to use real-world examples from its production systems in production training.', 'Meta says that its previous AI research infrastructure only leveraged open source and other publicly-available datasets.']
